In [1]:
import requests
from datetime import datetime, timedelta
from datetime import datetime, timedelta
import json
import requests
from time import sleep
from itertools import islice
from logging import getLogger
from tempfile import TemporaryDirectory
import os
import gc

In [2]:
logger = getLogger(__name__)

In [3]:
migration_status = {"overall": 0, "files_saved": 0, "migrated": 0}

def migration(start: str, end: str):
    """Process of migration tasks from CRM to CEM database

    Args:
        start (str): Start date in format "13.03.2020"
        end (str): End date in format "13.03.2020"
    """

    migration_start = datetime.now()

    limit = 10000

    TOKEN = ''
    URL = ''
    # 11 somewhat viable classes
    TAGS = {
            676: "refunds",
            675: "workflow_questions",
            674: "activation_data",
            673: "performance_stability_data",
            672: "luminar_x_data",
            670: "presale_marketing",
            669: "skylum_account",
            668: "billing_data",
            682: "spam",
            667: "licenses_downloads_data",
            680: "undefined"
           }
# All classes   
#     TAG = 678 
#     TAG = 678 # For Business
#     TAG = 677 # Partnership
#     TAG = 676 # Refunds
#     TAG = 675 # Workflow questions
#     TAG = 674 # Activation
#     TAG = 673 # Performance & stability
#     TAG = 672 # Luminar X
#     TAG = 671 # Feature request
#     TAG = 670 # Pre-sale, marketing
#     TAG = 669 # Skylum Account
#     TAG = 668 # BIlling
#     TAG = 682 # SPAM
#     TAG = 667 # License & downloads
#     TAG = 680 # Undefined

    LOCAL_DATA_FOLDER = "/Users/yaroslavkizyma/Desktop/json_data_exports_alternative"

    migration_status["overall"] = 0
    migration_status["files_saved"] = 0
    migration_status["migrated"] = 0

    migration_status["overall"] = int(
        # Status as a param - 1 processed, 0 - unprocessed, not needed for now
        # We need to receive ALL tickets instead
        # requests.get(
        #     f"{URL}?locale=skylum&language=en&status=1&start={start}&end={end}&quantity=true",
        #     headers={"Authorization": f"Bearer {TOKEN}"},
        # ).json()["tickets"]
        requests.get(
            f"{URL}?locale=skylum&language=en&start={start}&end={end}&quantity=true",
            headers={"Authorization": f"Bearer {TOKEN}"},
        ).json()["tickets"]
    )
    for tag, category in TAGS.items():  
        offset = 0
        while True:
            request_url = f"{URL}?limit={limit}&offset={offset}&locale=skylum&language=en&tag_id={tag}&start={start}&end={end}&trim=true"
            response = requests.get(request_url,
                                    headers={"Authorization": f"Bearer {TOKEN}"},
                                    )
            res_json = response.json()
            if not res_json["tickets"]:
                logger.warning(f"No new tickets")
                logger.warning(f"{request_url}")
                break
                
            dir_exists = os.path.exists(os.path.join(f"{LOCAL_DATA_FOLDER}", f"{category}"))
            if not dir_exists:
                os.makedirs(os.path.join(f"{LOCAL_DATA_FOLDER}", f"{category}"))
                
            with open(f"{LOCAL_DATA_FOLDER}/{category}/test_{offset}.json", "w") as _json:
                    json.dump(res_json["tickets"], _json)
                    logger.warning(f"Processed from: {offset}")
                    offset += 100
                    migration_status["files_saved"] += len(res_json["tickets"])
            migration_end = datetime.now()
            logger.warning(f"JSON saving time: {migration_end - migration_start}")
            gc.collect()
            logger.warning("migration done")
            migration_end = datetime.now()
            logger.warning(f"Migration time: {migration_end - migration_start}")


In [4]:
def initial_migrate_from_crm():
    migration(start="26.11.2021", end=datetime.utcnow().strftime("%d.%m.%Y"))
    sleep(10)
    migration_status["overall"] = 0
    migration_status["files_saved"] = 0
    migration_status["migrated"] = 0

In [5]:
initial_migrate_from_crm()

Processed from: 0
JSON saving time: 0:00:05.537202
migration done
Migration time: 0:00:05.558382
Processed from: 100
JSON saving time: 0:00:11.116913
migration done
Migration time: 0:00:11.135911
Processed from: 200
JSON saving time: 0:00:15.846010
migration done
Migration time: 0:00:15.864957
Processed from: 300
JSON saving time: 0:00:20.831114
migration done
Migration time: 0:00:20.849149
Processed from: 400
JSON saving time: 0:00:26.492160
migration done
Migration time: 0:00:26.510410
Processed from: 500
JSON saving time: 0:00:31.166601
migration done
Migration time: 0:00:31.186529
Processed from: 600
JSON saving time: 0:00:35.171384
migration done
Migration time: 0:00:35.191005
Processed from: 700
JSON saving time: 0:00:39.147416
migration done
Migration time: 0:00:39.172781
Processed from: 800
JSON saving time: 0:00:42.901599
migration done
Migration time: 0:00:42.919770
Processed from: 900
JSON saving time: 0:00:46.460361
migration done
Migration time: 0:00:46.478841
Processed fr

In [6]:
# Convert all data to dataframes & then to CSV files


In [7]:
import pandas as pd

In [79]:
json_folder_path = '/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/workflow_questions'
json_files = [pos_json for pos_json in os.listdir(json_folder_path) if pos_json.endswith('.json')]


In [ ]:
# new structure

In [12]:
def create_category_dataframe(category):
    jsons_data = pd.DataFrame(columns=['category', 'text'])
    
    json_folder_path = f'/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/{category}'
    json_files = [pos_json for pos_json in os.listdir(json_folder_path) if pos_json.endswith('.json')]

    
    # we need both the json and an index number so use enumerate()
    for js in json_files:
        with open(os.path.join(json_folder_path, js)) as json_file:
            json_text = json.load(json_file)
            for index, message in enumerate(json_text):
                text = message['message']
                category = category
                jsons_data.loc[index] = [category, text]
    return jsons_data

In [18]:
jsons_data_overall = pd.DataFrame(columns=['category', 'text'])

df_list = []
cat_list = ["refunds",
            "workflow_questions",
            "activation_data",
            "performance_stability_data",
            "luminar_x_data",
            "presale_marketing",
            "skylum_account",
            "billing_data",
            "spam",
            "licenses_downloads_data",
            "undefined"]
for category in cat_list:
    cat_df = create_category_dataframe(category)
    df_list.append(cat_df)
#     jsons_data_overall.append(cat_df)

# jsons_data_overall.append(df_list)

for idx, df in enumerate(df_list):
    df.to_csv(f'/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_{idx}.csv', index=False)

# jsons_data_overall.to_csv(f'/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_overall.csv', index=False)

In [20]:
df_activation = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_0.csv")
df_billing = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_1.csv")
df_licenses_downloads = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_2.csv")
df_luminar_x = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_3.csv")
df_performance_stability = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_4.csv")
df_presale_marketing = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_5.csv")
df_refunds = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_6.csv")
df_skylum_account = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_7.csv")
df_spam = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_8.csv")
df_undefined = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_9.csv")
df_workflow_questions = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/bert_data_10.csv")

In [21]:
jsons_data_overall = pd.DataFrame(columns=['category', 'text'])
jsons_data_overall = jsons_data_overall.append([df_activation, df_billing, df_licenses_downloads,df_luminar_x, 
                              df_performance_stability, df_presale_marketing,
                              df_refunds, df_skylum_account, df_spam,
                              df_undefined, df_workflow_questions])

In [22]:
jsons_data_overall.head()

,category,text
0,refunds,Please go ahead and issue the refund. I don't ...
1,refunds,"Hello, the first Transaction ID is listed as ..."
2,refunds,"Hi Tania,\n\nHere is the other invoice I recei..."
3,refunds,Thank you for your quick reply. \nIt's not som...
4,refunds,Name: Hans Wieldraaijer\nEmail: hans.wiel@telf...


In [ ]:
# old structure

In [ ]:
json_folder_path = '/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/workflow_questions'
json_files = [pos_json for pos_json in os.listdir(json_folder_path) if pos_json.endswith('.json')]

In [80]:
jsons_data = pd.DataFrame(columns=['message', 'tag'])

# we need both the json and an index number so use enumerate()
for js in json_files:
    with open(os.path.join(json_folder_path, js)) as json_file:
        json_text = json.load(json_file)
        for index, message in enumerate(json_text):
            text = message['message']
            tag = "workflow_questions"
            jsons_data.loc[index] = [text, tag]



In [81]:
len(jsons_data)

605

In [82]:
jsons_data.to_csv('/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/workflow_questions.csv', index=False)


In [12]:
# NOW TRY TO OPEN & APPEND ALL DATAFRAMES & COMBINE THEM INTO ONE

In [13]:
# pd.DataFrame().append([df1,df2, df3])

In [83]:
df_activation = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/activation_data.csv")
df_billing = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/billing_data.csv")
df_licenses_downloads = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/licenses_downloads_data.csv")
df_luminar_x = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/luminar_x_data.csv")
df_performance_stability = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/performance_stability_data.csv")
df_presale_marketing = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/presale_marketing.csv")
df_refunds = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/refunds.csv")
df_skylum_account = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/skylum_account.csv")
df_spam = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/spam.csv")
df_undefined = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/undefined.csv")
df_workflow_questions = pd.read_csv("/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/workflow_questions.csv")

In [84]:
jsons_data_overall = pd.DataFrame(columns=['message', 'tag'])
jsons_data_overall = jsons_data_overall.append([df_activation, df_billing, df_licenses_downloads,df_luminar_x, 
                              df_performance_stability, df_presale_marketing,
                              df_refunds, df_skylum_account, df_spam,
                              df_undefined, df_workflow_questions])

In [85]:
jsons_data_overall.head()

,message,tag
0,Name: tony\nEmail: hello@tonycannings.com\nMon...,activation_data
1,Name: Paul Boocock\nEmail: paulandjanetboocock...,activation_data
2,Name: william Peters\nEmail: gringopete55@gmai...,activation_data
3,Name: Bill Ransom\nEmail: billran1010@gmail.co...,activation_data
4,Name: diane dalli\nEmail: dianedalli@gmail.com...,activation_data


In [88]:
# Alternative encoding, for simplicity sake
jsons_data_overall.tag.unique()

array(['activation_data', 'billing_data', 'licenses_downloads_data',
       'luminar_x_data', 'performance_stability_data',
       'presale_marketing', 'refunds', 'skylum_account', 'spam',
       'undefined', 'workflow_questions'], dtype=object)

In [95]:
mapping = {'activation_data': 1, 'billing_data': 2, 'licenses_downloads_data': 3, 'luminar_x_data': 4,
           'performance_stability_data': 5, 'presale_marketing': 6, 'refunds': 7,
           'skylum_account': 8, 'spam': 9, 'undefined': 10, 'workflow_questions': 11}

In [96]:
jsons_data_overall=jsons_data_overall.replace({'tag': mapping})

In [97]:
jsons_data_overall.to_csv('/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/encoded_data_alternative.csv', index=False)

In [98]:
# Separate into train/test

In [99]:
from sklearn.model_selection import train_test_split

In [100]:
y = jsons_data_overall.tag
X = jsons_data_overall.drop('tag', axis=1)

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y)

In [102]:
type(y_train)

pandas.core.series.Series

In [103]:
output_train = pd.DataFrame(columns=['message', 'tag'])
output_test = pd.DataFrame(columns=['message', 'tag'])

In [104]:
output_train['message'] = X_train['message']
output_train['tag'] = y_train
output_test['message'] = X_test['message']
output_test['tag'] = y_test

In [105]:
output_train.to_csv('/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/encoded_data_alternative_train.csv', index=False)
output_test.to_csv('/Users/yaroslavkizyma/Desktop/json_data_exports_alternative/encoded_data_alternative_test.csv', index=False)

In [ ]:
# Old encoding, less effective

In [94]:
# encoded_stuff = pd.get_dummies(jsons_data_overall, columns=["tag"])

In [95]:
# encoded_stuff

,message,tag_Undefined,tag_activation,tag_billing,tag_licenses_downloads,tag_luminar_x,tag_performance_stability,tag_presale_marketing,tag_refunds,tag_skylum_account,tag_spam,tag_workflow_questions
0,Name: tony\nEmail: hello@tonycannings.com\nMon...,0,1,0,0,0,0,0,0,0,0,0
1,Name: Paul Boocock\nEmail: paulandjanetboocock...,0,1,0,0,0,0,0,0,0,0,0
2,Name: william Peters\nEmail: gringopete55@gmai...,0,1,0,0,0,0,0,0,0,0,0
3,Name: Bill Ransom\nEmail: billran1010@gmail.co...,0,1,0,0,0,0,0,0,0,0,0
4,Name: diane dalli\nEmail: dianedalli@gmail.com...,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
409,Name: Alessandro Catuogno\nEmail: info@alessan...,0,0,0,0,0,0,0,0,0,0,1
410,Name: jerome Fischer\nEmail: jsfischer@twc.com...,0,0,0,0,0,0,0,0,0,0,1
411,Hi Helena. Renaming did not help. \n\n> Am 11....,0,0,0,0,0,0,0,0,0,0,1
412,It was TOP thanks!\n\nSaludos Michel\n\n> On D...,0,0,0,0,0,0,0,0,0,0,1


In [96]:
# encoded_stuff.to_csv('/Users/yaroslavkizyma/Desktop/json_data_exports/encoded_data.csv', index=False)

In [97]:
# jsons_data_overall.to_csv('/Users/yaroslavkizyma/Desktop/json_data_exports/raw_data.csv', index=False)